In [15]:
from google.colab import drive
drive.mount('/content/drive')

# Cambiar el directorio de trabajo
%cd '/content/drive/My Drive/tfg/tfg/'

# Mostrar el directorio actual para confirmar el cambio
directorio_actual = %pwd
print("Directorio actual:", directorio_actual)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/tfg/tfg
Directorio actual: /content/drive/My Drive/tfg/tfg


In [16]:
import os
import pandas as pd
path_actual = os.getcwd()
subdirectorio = 'datas'
file_train = 'df_train.csv'
path_train = os.path.join(path_actual, subdirectorio, file_train)

In [17]:
import pandas as pd

df_train = pd.read_csv(path_train, index_col=None)


In [18]:
file_valid = 'df_valid.csv'
path_valid = os.path.join(path_actual, subdirectorio, file_valid)

df_valid = pd.read_csv(path_valid, index_col=None)

In [19]:
df_train = df_train.drop(['trueE'], axis=1)
df_valid = df_valid.drop(['trueE'], axis=1)


Este modulo será el encargado de entrenar los modelos de machine learning y ver los resultados que muetran. Además elegiremos los mejores hiperparametros para los modelos ya que estos modelos serán cargados de la biblioteca Scikit-learn.

Comprobaremos las diferencia de cada modelo para ver que modelo da mejores resultados.

Los modelos que vamos a usar son los siguientes:

*   **Regresión logistica:** Es un modelo lineal que es simple pero eficaz para problemas de clasificación binaria. Este modelo nos dará el punto de partida.

*   **Arboles de decisión:** Es un modelo versatil que puede usarse tanto para clasificación como para regresión. Los arboles de decisión son fáciles de interpretar y pueden capturar relaciones no lineales entre caracterisiticas.

*   **Random Forest:** Un ensamble de arboles de decisión. Es más robusto y preciso que un árbol unico de decisión debido a que reduce el riesgo de sobreajuste mediante el promedio de múltiples árboles

*   **SVM:** Es efectivo en espación de alta dimensión y casos donde el número de dimensiones es mayor que el número de muestras. Es útil tanto para la clasificación lineal como no lineal mediante el uso de diferentes kerneles.

*   **GBM:** Es una técnica de ensamble que construye modelos de forma secuencial, cada nuevo modelo corrige errores del modelo anterior.

*  **k-NN:** Es un modelo simple que clasifica puntos basándose en la mayoría de votos de sus k vecinos más cercanos. Es útil cuando las relaciones entre caracteristicas son complejas y no lineales.

*  **XGBoost:** es una implementación optimizada de árboles de decisión aplicados al algoritmo de Gradient Boosting. Es muy eficaz por su eficiencia en la ejecución.

*   **LightGBM:** es otro algoritmo de Gradient Boosting desarrollado por Microsoft que utiliza árboles basados en histogramas para el modelado.Utiliza la técnica de construcción de árboles basada en histogramas, que agrupa valores continuos en bins discretos



In [20]:
import numpy as np
import pandas as pd

def filtrado_datos(df, N):
    kaones = []
    labels = []

    df_sorted = df.sort_values(by=['eventID', 'hitTime'], ascending=[True, False])

    for eventID, grupo in df_sorted.groupby('eventID'):
        pdgCodes = grupo['PDGcode'].unique()

        for pdgCode in pdgCodes:
            grupo_filtrado = grupo[grupo['PDGcode'] == pdgCode]
            grupo_ordenado = grupo_filtrado.head(N)

            # Inicializar arrays para el padding
            hitX_padded = np.zeros(N)
            hitY_padded = np.zeros(N)
            hitZ_padded = np.zeros(N)
            hitInteg_padded = np.zeros(N)

            # Separar y aplicar padding a los valores de hitX, hitY, hitZ, hitInteg
            hitX_padded[:len(grupo_ordenado['hitX'])] = grupo_ordenado['hitX']
            hitY_padded[:len(grupo_ordenado['hitY'])] = grupo_ordenado['hitY']
            hitZ_padded[:len(grupo_ordenado['hitZ'])] = grupo_ordenado['hitZ']
            hitInteg_padded[:len(grupo_ordenado['hitInteg'])] = grupo_ordenado['hitInteg']

            # Concatenar los valores ya con el padding aplicado
            hit_values_reorganized = np.concatenate([hitX_padded, hitY_padded, hitZ_padded, hitInteg_padded])

            kaones.append(hit_values_reorganized)
            labels.append(pdgCode)

    return np.array(kaones), np.array(labels)


In [21]:
!pip install xgboost lightgbm

Hacemos un import de todos los modelos que vamos a usar

In [22]:
from sklearn.linear_model import LogisticRegression # Regresión logistica
from sklearn.tree import DecisionTreeClassifier # Arbol de decision
from sklearn.ensemble import RandomForestClassifier # Random Forest
from sklearn.svm import SVC # SVM
from sklearn.ensemble import GradientBoostingClassifier # Gradient Boosting Machine
from sklearn.neighbors import KNeighborsClassifier # k-NN
from xgboost import XGBClassifier # XGBoost
from lightgbm import LGBMClassifier # LightGBM

Hacemos un import de todas las metricas que vamos a usar

In [23]:
from sklearn.metrics import accuracy_score

Como hemos indicado en el notebook `estadisticas.ipynb`, para tratar de encontrar el mejor valor de N realizaremos varios entrenamientos con distintos valores de N para ver el comporamiento de los modelos. El valor de N ira en el intervalo [400,800] en saltos de 40. A continuación crearemos el array.

In [24]:
valores_N = np.arange(400,801,40)
valores_N

array([400, 440, 480, 520, 560, 600, 640, 680, 720, 760, 800])

> Regresión logisitica (LogisticRegression)


*   **Penalty=l2**: debido a que todas las caracteristicas son importantes.
*   List item



In [26]:
RL = LogisticRegression(penalty='l2', solver='lbfgs')

matriz_train, labels_train = filtrado_datos(df_train, 400)
matriz_valid, labels_valid = filtrado_datos(df_valid, 400)


RL.fit(matriz_train, labels_train)
y_predic = RL.predict(matriz_valid)
accuracy_LogisticRegresion = accuracy_score(labels_valid, y_predic)
print(f"Accuracy: {accuracy_LogisticRegresion}")

Accuracy: 0.8228361424407553


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [29]:
accuracys_LogisticRegresion = []

for N in valores_N:

  RL = LogisticRegression(penalty='l2', solver='lbfgs', random_state=42, max_iter=500, multi_class = 'ovr')

  matriz_train, labels_train = filtrado_datos(df_train, N)
  matriz_valid, labels_valid = filtrado_datos(df_valid, N)

  RL.fit(matriz_train, labels_train)
  y_predic = RL.predict(matriz_valid)
  accuracy = accuracy_score(labels_valid, y_predic)
  accuracys_LogisticRegresion.append(accuracy)



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [31]:
accuracys_LogisticRegresion

[0.8314535546825498,
 0.8365226207071347,
 0.832720821188696,
 0.8365226207071347,
 0.8380433405145102,
 0.8400709669243442,
 0.8405778735268027,
 0.8410847801292611,
 0.8370295273095932,
 0.8393106070206564,
 0.8360157141046762]